In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

client_credentials_manager = SpotifyClientCredentials(
        "26399552a8ce4d1285397254189cac50",
        "fdacfffa2dd34dbeb127dedb459f7ea3")
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [2]:
def getTrackFeatures(trackURI):
    ger_data = pd.DataFrame()
    for track in trackURI:
        audio = sp.audio_analysis(track)
        segments_data =  pd.DataFrame(audio["segments"])
        pitch = segments_data.pitches.apply(pd.Series)
        pitch.columns = ["p" + str(i) for i in range(1,13)]
        timbre = segments_data.timbre.apply(pd.Series)
        timbre.columns = ["t" + str(i) for i in range(1,13)]
        segments_data = segments_data.drop(["pitches", "timbre", "loudness_end"],
                                           axis = 1)
        segments_data = segments_data.join([pitch, timbre])
        segments_data["track_id"] = track
        ger_data = ger_data.append(segments_data)
    return ger_data

In [ ]:
Africa = ['DZ', 'EG', 'MA', 'ZA', 'TN']
Asia = ['BH', 'HK', 'IN', 'ID', 'IL', 'JP', 'JO', 'KW', 
        'LB', 'MY', 'OM', 'PS', 'PH', 'QA', 'SA', 'SG', 
        'TW', 'TH', 'AE', 'VN']
Europe = ['AD', 'AT', 'BE', 'BG', 'CY', 'CZ', 'DK', 'EE', 
          'FI', 'FR', 'DE', 'GR', 'HU', 'IS', 'IE', 'IT', 
          'LV', 'LI', 'LT', 'LU', 'MT', 'MC', 'NL', 'NO', 
          'PL', 'PT', 'RO', 'SK', 'ES', 'SE', 'CH', 'TR', 'GB']
NorthAmerica = ['CA', 'CR', 'DO', 'SV', 'GT', 'HN', 'MX', 'NI', 'PA', 'US']
SouthAmerica = ['AR', 'BO', 'BR', 'CL', 'CO', 'EC', 'PY', 'PE', 'UY']
Oceania = ['AU', 'NZ']

countriesOfInterest = Africa + Asia + Oceania
print(len(countriesOfInterest))
for year in range(2019,2000, -1):
    tmstamp = str(year) + "-06-15T12:00:00"
    for countryCode in countriesOfInterest:
        country = sp.featured_playlists(country = countryCode, limit = 50)
        plist = country["playlists"]["items"]
        print(countryCode, "has", len(plist), "playlists")
        plist = pd.DataFrame.from_dict(plist)
        plist.to_csv("Raw Playlist Data\\" + countryCode + "_" + str(year) + ".csv", index = False)
        for uri in plist["uri"]:
            currplist = sp.playlist(uri)
            trackURI = []
            for track in currplist["tracks"]["items"]:
                try:
                    trackURI = trackURI + [track["track"]["uri"]]
                except:
                    continue
            print("Getting track features for", uri, "in", countryCode)
            trackData = getTrackFeatures(trackURI)
            trackData.to_csv("Raw Track Data\\" + countryCode + "_" + str(year) + "_" + uri[17:] + ".csv", index = False)

27
DZ has 16 playlists
Getting track features for spotify:playlist:37i9dQZF1DWZKucvJjqs4a in DZ
Getting track features for spotify:playlist:37i9dQZF1DWYHO8PTSQ9fM in DZ
Getting track features for spotify:playlist:37i9dQZF1DX1X23oiQRTB5 in DZ
Getting track features for spotify:playlist:37i9dQZF1DX68x6hand0cN in DZ
Getting track features for spotify:playlist:37i9dQZF1DXd3AhRYJnfcl in DZ
Getting track features for spotify:playlist:37i9dQZF1DX0ysO3y1BFXN in DZ
Getting track features for spotify:playlist:37i9dQZF1DX5cO1uP1XC1g in DZ
Getting track features for spotify:playlist:37i9dQZF1DX1fzTwl8xjJf in DZ
Getting track features for spotify:playlist:37i9dQZF1DX4rmnjMBxfCv in DZ
Getting track features for spotify:playlist:37i9dQZF1DXaHuo0R6jTxZ in DZ
Getting track features for spotify:playlist:37i9dQZF1DWYFV1CnuXO8v in DZ
Getting track features for spotify:playlist:37i9dQZF1DWUQM3rmTXpBR in DZ
Getting track features for spotify:playlist:37i9dQZF1DXdcjKKBPqh1t in DZ
Getting track features for s

Getting track features for spotify:playlist:37i9dQZF1DX9Sg2GTUELDO in HK
Getting track features for spotify:playlist:37i9dQZF1DX0x36cwEyOTG in HK
Getting track features for spotify:playlist:37i9dQZF1DWT1y71ZcMPe5 in HK
Getting track features for spotify:playlist:37i9dQZF1DWX83CujKHHOn in HK
Getting track features for spotify:playlist:37i9dQZF1DX4WYpdgoIcn6 in HK
IN has 12 playlists
Getting track features for spotify:playlist:37i9dQZF1DX4xuWVBs4FgJ in IN
Getting track features for spotify:playlist:37i9dQZF1DX6GwdWRQMQpq in IN
Getting track features for spotify:playlist:37i9dQZF1DX82pCGH5USnM in IN
Getting track features for spotify:playlist:37i9dQZF1DWY4xHQp97fN6 in IN
Getting track features for spotify:playlist:37i9dQZF1DX9XIFQuFvzM4 in IN
Getting track features for spotify:playlist:37i9dQZF1DXbYM3nMM0oPk in IN
Getting track features for spotify:playlist:37i9dQZF1DX1rVvRgjX59F in IN
Getting track features for spotify:playlist:37i9dQZF1DXcBWIGoYBM5M in IN
Getting track features for spot

Getting track features for spotify:playlist:37i9dQZF1DWY8U6Zq7nvbE in MY
Getting track features for spotify:playlist:37i9dQZF1DX0s5kDXi1oC5 in MY
Getting track features for spotify:playlist:37i9dQZF1DXcA6dRp8rwj6 in MY
Getting track features for spotify:playlist:37i9dQZF1DX3Ogo9pFvBkY in MY
OM has 18 playlists
Getting track features for spotify:playlist:37i9dQZF1DWWgW3m1g7KmT in OM
Getting track features for spotify:playlist:37i9dQZF1DX905nNo3hBpt in OM
Getting track features for spotify:playlist:37i9dQZF1DX5lckay3OqOG in OM
Getting track features for spotify:playlist:37i9dQZF1DX2siSxlNkZrf in OM
Getting track features for spotify:playlist:37i9dQZF1DWU486KSiznWZ in OM
Getting track features for spotify:playlist:37i9dQZF1DXaFm6IhSHlyi in OM
Getting track features for spotify:playlist:37i9dQZF1DXd3AhRYJnfcl in OM
Getting track features for spotify:playlist:37i9dQZF1DXa7FSukIBTlS in OM
Getting track features for spotify:playlist:37i9dQZF1DX5cO1uP1XC1g in OM
Getting track features for spot

In [ ]:
for year in range(2019,2000, -1):
    print(year)